In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Setup the driver. This one uses Chrome with the ChromeDriver executable in the PATH
driver = webdriver.Chrome()

# Open the webpage
driver.get("https://wolnelektury.pl/katalog/autor/adam-mickiewicz/")


book_links = []

# Wait for the page to load
time.sleep(3)
# Get all of l-books__item book-container-activator articles and extract the first a href link from them
# Find all the article elements that contain books
book_articles = driver.find_elements(By.CSS_SELECTOR, "article.l-books__item.book-container-activator")
print(f"Found {len(book_articles)} articles")
# Loop through each article to extract information
for article in book_articles:
    # Get the link thats inside the figure element
    link = article.find_element(By.CSS_SELECTOR, "figure a")
    # Get the title thats inside the figure element
    book_links.append(link.get_attribute('href'))


# Close the driver
driver.quit()


Found 56 articles


In [15]:
book_links

['https://wolnelektury.pl/katalog/lektura/ballady-i-romanse/',
 'https://wolnelektury.pl/katalog/lektura/mickiewicius-baltas-karzigys/',
 'https://wolnelektury.pl/katalog/lektura/sonety-odeskie-blogoslawienstwo-z-petrarki/',
 'https://wolnelektury.pl/katalog/lektura/mickiewicz-bajki-chlop-i-zmija/',
 'https://wolnelektury.pl/katalog/lektura/czaty/',
 'https://wolnelektury.pl/katalog/lektura/mickiewicz-do-d-d/',
 'https://wolnelektury.pl/katalog/lektura/sonety-odeskie-do-d-d-elegia/',
 'https://wolnelektury.pl/katalog/lektura/mickiewicz-do-joachima-lelewela/',
 'https://wolnelektury.pl/katalog/lektura/sonety-odeskie-do-m/',
 'https://wolnelektury.pl/katalog/lektura/sonety-odeskie-do-marii-p/',
 'https://wolnelektury.pl/katalog/lektura/mickiewicz-do-maryi-lempickiej/',
 'https://wolnelektury.pl/katalog/lektura/mickiewicz-do-maryi-szymanowskiej/',
 'https://wolnelektury.pl/katalog/lektura/mickiewicz-do-matki-polki/',
 'https://wolnelektury.pl/katalog/lektura/mickiewicz-do-mojego-cicerone/

In [56]:
def crawl_link(link, crawled_links=None):
    if crawled_links is None:
        crawled_links = []

    print(f"Crawling {crawled_links}")
    if link in crawled_links:
        return []

    print(f"Crawling {link}")
    driver = None
    return_links = []

    try:
        driver = webdriver.Chrome()
        driver.get(link)
        crawled_links.append(link)

        a_elements = driver.find_elements(By.CSS_SELECTOR, "ul.l-aside__zbiory")
        if a_elements:
            a_elements = a_elements[0].find_elements(By.CSS_SELECTOR, "a")
            for a in a_elements:
                href = a.get_attribute('href')
                if href not in crawled_links:
                    return_links.extend(crawl_link(href, crawled_links))

        button = driver.find_element(By.ID, "ebook")
        if button:
            button.click()

        download_txt = driver.find_element(By.CSS_SELECTOR, "a[href*='txt']")
        if download_txt:
            download_txt.click()
            return_links.append(driver.current_url)

    except Exception as e:
        print(f"Error: {e}")
    finally:
        if driver:
            driver.quit()

    return return_links


In [62]:

txt_file_links = []

for link in book_links:
    txt_file_links += crawl_link(link)

print(txt_file_links)


Crawling []
Crawling https://wolnelektury.pl/katalog/lektura/ballady-i-romanse/
Crawling ['https://wolnelektury.pl/katalog/lektura/ballady-i-romanse/']
Crawling https://wolnelektury.pl/katalog/lektura/mickiewicz-ballady-i-romanse-pierwiosnek/
Crawling ['https://wolnelektury.pl/katalog/lektura/ballady-i-romanse/', 'https://wolnelektury.pl/katalog/lektura/mickiewicz-ballady-i-romanse-pierwiosnek/']
Crawling https://wolnelektury.pl/katalog/lektura/ballady-i-romanse-romantycznosc/
Crawling ['https://wolnelektury.pl/katalog/lektura/ballady-i-romanse/', 'https://wolnelektury.pl/katalog/lektura/mickiewicz-ballady-i-romanse-pierwiosnek/', 'https://wolnelektury.pl/katalog/lektura/ballady-i-romanse-romantycznosc/']
Crawling https://wolnelektury.pl/katalog/lektura/ballady-i-romanse-switez/
Crawling ['https://wolnelektury.pl/katalog/lektura/ballady-i-romanse/', 'https://wolnelektury.pl/katalog/lektura/mickiewicz-ballady-i-romanse-pierwiosnek/', 'https://wolnelektury.pl/katalog/lektura/ballady-i-ro

In [63]:
txt_file_links

['https://wolnelektury.pl/media/book/txt/mickiewicz-dudarz.txt',
 'https://wolnelektury.pl/media/book/txt/ballady-i-romanse-lilje.txt',
 'https://wolnelektury.pl/media/book/txt/mickiewicz-tukaj.txt',
 'https://wolnelektury.pl/media/book/txt/ballady-i-romanse-pani-twardowska.txt',
 'https://wolnelektury.pl/media/book/txt/ballady-i-romanse-rekawiczka.txt',
 'https://wolnelektury.pl/media/book/txt/mickiewicz-ballady-i-romanse-to-lubie.txt',
 'https://wolnelektury.pl/media/book/txt/mickiewicz-ballady-i-romanse-do-przyjaciol.txt',
 'https://wolnelektury.pl/media/book/txt/mickiewicz-ballady-i-romanse-kurhanek-maryli.txt',
 'https://wolnelektury.pl/media/book/txt/ballady-i-romanse-powrot-taty.txt',
 'https://wolnelektury.pl/media/book/txt/ballady-i-romanse-rybka.txt',
 'https://wolnelektury.pl/media/book/txt/ballady-i-romanse-switezianka.txt',
 'https://wolnelektury.pl/media/book/txt/ballady-i-romanse-switez.txt',
 'https://wolnelektury.pl/media/book/txt/ballady-i-romanse-romantycznosc.txt',


In [61]:
# download the files and save them to books
import requests
import os
for link in txt_file_links:
    file_name = os.path.join("books", link.split("/")[-1])
    if not os.path.exists(file_name):
        response = requests.get(link)
        with open(file_name, 'wb') as f:
            f.write(response.content)
